In [1]:
import skimage.io as io
import time
import numpy as np
import cv2
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from ChainCodesFeatures import *
from PolygonFeatures import * 
from WindowFeatures import *
from windows import *

In [9]:
def Feature_Extractor(img):
    
    st = time.time()
    edged = cv2.Canny(img, 30, 200)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    en = time.time()
#     print("Contours : ",(en-st)*1000,"ms")
    
    st = time.time()
    windows, clusters, clusteredWindows = getClusteredWindows2(img,13,10)
    en = time.time()
#     print("Kesho : ",(en-st)*1000,"ms")
    
    st = time.time()
    F1,F2,F3,F4,F5,F7,F8,F9 = Chain_Codes_Feature_Extractor(img,contours,clusteredWindows)
    en = time.time()
#     print("Ramzy : ",(en-st)*1000,"ms")
    
    st = time.time()
    f10, f11, f12, f13, f14 = PolygonFeatures(contours)
    en = time.time()
#     print("Ibrahim Lefta : ",(en-st)*1000,"ms")
    
    st = time.time()
    Windows = list()
    for i in range(clusteredWindows.shape[0]):
        pass
        #Windows  += list(WindowFeatures(clusteredWindows[i].astype(np.uint8)))
    en = time.time()
#     print("October : ",(en-st)*1000,"ms")
    
    Features = list(F1)+list(F2)+list(F3)+list(F4)+list(F5)+list(F7)+list(F8)+list(F9)+list(f10)+list(f11)+list(f12)+list(f13)+list(f14)+Windows
    return Features
    

In [5]:
img = cv2.imread("TestCases/class1_0.png", cv2.IMREAD_GRAYSCALE)
Features_1 = Feature_Extractor(img)
img = cv2.imread("TestCases/class0_0.png", cv2.IMREAD_GRAYSCALE)
Features_0 = Feature_Extractor(img)
img = cv2.imread("TestCases/class1_1.png", cv2.IMREAD_GRAYSCALE)
Features_pred_1 = Feature_Extractor(img)
img = cv2.imread("TestCases/class0_1.png", cv2.IMREAD_GRAYSCALE)
Features_pred_0 = Feature_Extractor(img)

Contours :  61.83505058288574 ms
Kesho :  1838.0844593048096 ms
Ramzy :  1178.8434982299805 ms
Ibrahim Lefta :  1555.8717250823975 ms
October :  4.954099655151367 ms
Contours :  46.87643051147461 ms
Kesho :  3302.168607711792 ms
Ramzy :  770.9746360778809 ms
Ibrahim Lefta :  980.893611907959 ms
October :  5.021572113037109 ms
Contours :  56.88166618347168 ms


KeyboardInterrupt: 

In [4]:
features_test = np.array([Features_0,Features_1])
features_train = np.array([Features_pred_0,Features_pred_1])

In [5]:
gnb = GaussianNB()
gnb.fit(features_train,[0,1])

GaussianNB(priors=None, var_smoothing=1e-09)

In [6]:
result = gnb.predict(features_test)
actual_label = [0,1]
Acc = (actual_label==result).sum()/len(actual_label)
print("Accuracy : ",Acc*100,"%")

Accuracy :  100.0 %


In [3]:
with open("forms.txt") as h:
    lines = h.readlines()[16:]

In [4]:
freq = np.zeros(680, dtype=int)
formWriter = {}
for i in lines:
    line = i.split(' ')
    key = line[1]
    if key in formWriter.keys():
        formWriter[key].append(line[0])
    else:
        formWriter[key] = [line[0]]

In [5]:
freq = [x for x, l in formWriter.items() if len(l)>2]

In [ ]:
acc = 0
for i in range(5):
    keys = np.random.randint(len(freq), size=3)
    train = []
    test = []
    for k in keys:
        forms = formWriter[freq[k]]
        idxes = np.random.randint(len(forms), size=3)
        train.append(forms[idxes[0]])
        train.append(forms[idxes[1]])
        test.append(forms[idxes[2]])
    testWriter = np.random.randint(len(test))
    testImg = test[testWriter]
    
    trainFeatures = []
    Y = []
    for i in range(len(train)):
        img = cv2.imread("forms/"+train[i]+'.png', cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        f = Feature_Extractor(img)
        trainFeatures.append(f)
        Y.append(i//2)
#         print(i//2)
    gnb = SVC()
    gnb.fit(trainFeatures, Y)
    
    img = cv2.imread("forms/"+testImg+'.png', cv2.IMREAD_GRAYSCALE)
    testFeatures = Feature_Extractor(img)                     
    result = gnb.predict([testFeatures])
    if result[0] == testWriter:
        acc += 1
    print(result, testWriter)


In [12]:
result, testWriter

(array([0]), 1)

In [18]:
Y

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5]

In [10]:
result.shape

(1,)

In [11]:
print(train, test)

['e04-000', 'c04-008', 'f07-039b', 'f07-039b', 'n04-107', 'n04-100'] ['c04-013', 'f07-036', 'n04-100']


In [14]:
len(f)

1463

In [10]:
acc = 0
total = 0
for itr in range(100):
    keys = np.random.randint(len(freq), size=3)
    train = []
    test = []
    for k in keys:
        forms = formWriter[freq[k]]
        forms = [freq[k]+'-'+'{:03d}'.format(s)+'.jpg' for s in range(len(forms))]
#         print(forms)
        idxes = np.random.randint(len(forms), size=3)
        
        train.append(forms[idxes[0]])
        train.append(forms[idxes[1]])
        test.append(forms[idxes[2]])
    testWriter = np.random.randint(len(test))
    testImg = test[testWriter]
    
    trainFeatures = []
    for i in train:
        img = cv2.imread('train/'+i, cv2.IMREAD_GRAYSCALE)
        f = Feature_Extractor(img)
        trainFeatures.append(f)
    gnb = SVC()
    gnb.fit(trainFeatures, [0, 0, 1, 1, 2, 2])
    
    img = cv2.imread("train/"+testImg, cv2.IMREAD_GRAYSCALE)
    testFeatures = Feature_Extractor(img)                     
    result = gnb.predict([testFeatures])
    if result[0] == testWriter:
        acc += 1
    total +=1
    print(total,acc, "Accuracy : ",acc*100/total,"%")
    print(result, testWriter)


1 1 Accuracy :  100.0 %
[1] 1
2 2 Accuracy :  100.0 %
[0] 0
3 3 Accuracy :  100.0 %
[1] 1
4 4 Accuracy :  100.0 %
[0] 0
5 5 Accuracy :  100.0 %
[1] 1
6 5 Accuracy :  83.33333333333333 %
[1] 2
7 6 Accuracy :  85.71428571428571 %
[2] 2
8 7 Accuracy :  87.5 %
[0] 0
9 8 Accuracy :  88.88888888888889 %
[1] 1
10 9 Accuracy :  90.0 %
[0] 0
11 9 Accuracy :  81.81818181818181 %
[1] 0
12 10 Accuracy :  83.33333333333333 %
[1] 1
13 11 Accuracy :  84.61538461538461 %
[2] 2
14 12 Accuracy :  85.71428571428571 %
[1] 1
15 13 Accuracy :  86.66666666666667 %
[2] 2


TypeError: `arr_in` must be a numpy ndarray

In [9]:
acc

28

In [10]:
itr

32

In [13]:
print(img)

None
